# Get BM25 recommendation

In [1]:
import logging
logging.basicConfig(filename='error.log', level=logging.ERROR)
from datetime import datetime
FILES = [
    "grandmaster_nl_pl_only_plot.json",
    "master_nl_pl_only_plot.json",
    "expert_nl_pl_only_plot.json",
]

CLASS_NAME = {
    "grandmaster": "GrandMasterCode",
    "master": "MasterCode",
    "expert": "ExpertCode",
}


Weaviate Instance

In [2]:
import json
from json import loads
import weaviate
from elasticsearch import Elasticsearch


weaviate_client = weaviate.Client("http://202.151.177.149:81")  # Replace with your endpoint
some_objects = weaviate_client.data_object.get()
if (json.dumps(some_objects)):
    print(True)
else:
    print(False)

print(json.dumps(some_objects))


elastic_client = Elasticsearch("http://202.151.177.154:9200")

response = str(elastic_client.info())
# print(response)

True
{"deprecations": null, "objects": [{"class": "ExpertCode", "creationTimeUnix": 1680809768520, "id": "00004e69-955d-4519-92db-581b8222e7c7", "lastUpdateTimeUnix": 1680809768520, "properties": {"code": "# Reference: https://www.kaggle.com/code/yamsam/ashrae-highway-kernel-route4\n\n# i'm now using my leak data station kernel to shortcut.\nleak_df = pd.read_feather('../input/ashrae-leak-data-station/leak.feather')\n\nleak_df.fillna(0, inplace=True)\nleak_df = leak_df[leak_df.timestamp.dt.year > 2016]\nleak_df.loc[leak_df.meter_reading < 0, 'meter_reading'] = 0 # remove large negative values\nleak_df = leak_df[leak_df.building_id!=245]\n\nsample_submission.loc[sample_submission.meter_reading < 0, 'meter_reading'] = 0\n\ntest_df['pred'] = sample_submission.meter_reading\n\nleak_df = leak_df.merge(test_df[['building_id', 'meter', 'timestamp', 'pred', 'row_id']], left_on = ['building_id', 'meter', 'timestamp'], right_on = ['building_id', 'meter', 'timestamp'], how = \"left\")\nleak_df = 

In [9]:
def getMLRecommendation(text: str, target_class: str) -> dict:
    # md_text = obj['markdown']
    # cur_class = "grandmaster"
    near_text = {"concepts": [text]}
    fetched = (weaviate_client.query
                      .get(CLASS_NAME[target_class], ["code"])
                      .with_near_text(near_text)
                      .with_limit(1)
                      .do()
                      )
    data = fetched['data']['Get'][CLASS_NAME[target_class]]
    return data[0]

def getElasticQuery(query, type=["base", "processed"]):
    if (type == "processed"):
        result = {
        "query": {
            "match": {
                "processed": {
                    "query": query
                }
            }
        }
    }
    else:
        result = {
        "query": {
            "match": {
                "markdown": {
                    "query": query
                }
            }
        }
    }
    return result 

def getElasticRecommendation(index, queryBody):
    response = elastic_client.search(index=index, body=queryBody)

    if response and response["hits"]["hits"]:
        result = response["hits"]["hits"][0]["_source"]
        return result
    return []

Load recommendation result into JSON files

In [4]:
for file_name in FILES:
    with open(f'data/{file_name}', 'r') as file:
        data = json.load(file)
        data_rank = file_name[:-21]

        data_length = len(data)
        testing_accumulate = []
        count = 0
        for row in data:
            markdown = "".join(row['markdown'])
            
            if (not markdown):
                count =  count + 1
        print(f'Total Empty Markdown in rank {data_rank} = {count}')

Total Empty Markdown in rank grandmaster = 22
Total Empty Markdown in rank master = 116
Total Empty Markdown in rank expert = 528


In [10]:
for file_name in FILES:
    with open(f'data/{file_name}', 'r') as file:
        data = json.load(file)
        data_rank = file_name[:-21]

        data_length = len(data)
        testing_accumulate = []
        count = 0
        

        for row in data:
            if (count>=1):
                break
            
            markdown = "".join(row['markdown'])
            
            try:
                temp_result = {
                    "original_md": markdown,
                    "original_code": row['code'],
                    "count": count,
                    "data_rank": data_rank
                }

                es_query = getElasticQuery(markdown)
                es_result = getElasticRecommendation(data_rank, es_query)
                es_result = json.dumps(es_result, indent=4)
                print(es_result)

                # # Get ML Recommendation
                # recommended_code_ml = getMLRecommendation(markdown, data_rank)
                # temp_result["recommended_code_ml"] = recommended_code_ml['code']
            
                # # Get Elastic Recommendation
                # query_base = getElasticQuery(markdown)
                # query_processed = getElasticQuery(markdown, "processed")

                # recommended_code_es_base = getElasticRecommendation(data_rank, query_base)
                # recommended_code_es_processed = getElasticRecommendation(data_rank, query_processed)

                # temp_result['recommended_code_es_base'] = recommended_code_es_base
                # temp_result['recommended_code_es_processed'] = recommended_code_es_processed

                # # Append to the collection
                # testing_accumulate.append(temp_result)

                # print(f'Count: {count}/{data_length}')
                
            except Exception as e:
                print(f'[{datetime.now().strftime("%d/%m/%y %H:%M:%S")}] Skipping item #{count} due to an error.')
                logging.error(f"Error occurred for item: {markdown}\nError message: {str(e)}\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n")
                continue
            count =  count + 1
            
        
        # with open(f"recommendation result/{data_rank}_result.json", "w") as file:
        #     json.dump(testing_accumulate, file)
        #     print(f"successfully write to: {data_rank}_result.json ")

C:\Users\Veerakit_Admin\AppData\Local\Temp\ipykernel_4200\4069455083.py:38: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = elastic_client.search(index=index, body=queryBody)


{
    "markdown": [
        "# Acknowledgements\n\n1. [M5 Forecasting - Starter Data Exploration](https://www.kaggle.com/robikscube/m5-forecasting-starter-data-exploration) ~ by Rob Mulla\n2. [EDA and Baseline Model](https://www.kaggle.com/rdizzl3/eda-and-baseline-model) ~ by RDizzl3\n3. [How to Create an ARIMA Model for Time Series Forecasting in Python](https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/) ~ by Machine Learning Mastery\n4. [7 methods to perform Time Series forecasting (with Python codes)](https://www.analyticsvidhya.com/blog/2018/02/time-series-forecasting-methods/) ~ by Analytics Vidhya\n5. [Economics for the IB Diploma](https://www.cambridge.org/core/books/economics-for-the-ib-diploma/1918CF16A8FC979AAB19951A487DCB1C) ~ by Ellie Tragakes\n6. [Prophet](https://facebook.github.io/prophet/) ~ by Facebook",
        "# Contents\n\n* [<font size=4>The dataset</font>](#1)\n\n\n* [<font size=4>EDA</font>](#2)\n    * [Preparing the ground](#2.1)

In [ ]:
for file_name in FILES:
    with open(f'data/{file_name}', 'r') as file:
        data = json.load(file)
        data_rank = file_name[:-21]

        data_length = len(data)
        testing_accumulate = []
        count = 0
        for row in data:
            markdown = "".join(row['markdown'])
            
            try:
                # temp_result = {
                #     "original_md": markdown,
                #     "original_code": row['code'],
                #     "count": count,
                #     "data_rank": data_rank
                # }

                temp_result = {
                    "running_count_label": count,
                    "data_rank": data_rank,
                    # Template for insert
                    "original": {
                        "markdown": "",
                        "code": "",
                    },
                    "mlRecommendation": {
                        "code": ""
                    },
                    "bm25Recommendation": {
                        "markdown": "",
                        "processed_markdown": "",
                        "code": "",
                    },
                    "bm25ProcessedRecommendation": {
                        "markdown": "",
                        "processed_markdown": "",
                        "code": ""
                    }
                }

                # Get ML Recommendation
                recommended_code_ml = getMLRecommendation(markdown, data_rank)
                temp_result["mlRecommendation"]["code"] = recommended_code_ml['code']
            
                # Get Elastic Recommendation
                query_base = getElasticQuery(markdown)
                query_processed = getElasticQuery(markdown, "processed")

                recommended_code_es_base = getElasticRecommendation(data_rank, query_base)
                recommended_code_es_processed = getElasticRecommendation(data_rank, query_processed)

                temp_result["bm25Recommendation"]["markdown"] = recommended_code_es_base['markdown']
                temp_result["bm25Recommendation"]["processed_markdown"] = recommended_code_es_base['processed']
                temp_result["bm25Recommendation"]["code"] = recommended_code_es_base['code']

                temp_result["bm25ProcessedRecommendation"]["markdown"] = recommended_code_es_base['markdown']
                temp_result["bm25ProcessedRecommendation"]["processed_markdown"] = recommended_code_es_base['processed']
                temp_result["bm25ProcessedRecommendation"]["code"] = recommended_code_es_base['code']

                # temp_result['recommended_code_es_base'] = recommended_code_es_base['code']
                # temp_result['recommended_code_es_processed'] = recommended_code_es_processed['code']

                # Append to the collection
                testing_accumulate.append(temp_result)

                print(f'Count: {count}/{data_length}')
                
            except Exception as e:
                print(f'[{datetime.now().strftime("%d/%m/%y %H:%M:%S")}] Skipping item #{count} due to an error.')
                logging.error(f"Error occurred for item: {markdown}\nError message: {str(e)}\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n")
                continue
            count =  count + 1
        
        with open(f"recommendation result/{data_rank}_result.json", "w") as file:
            json.dump(testing_accumulate, file)
            print(f"successfully write to: {data_rank}_result.json ")

Comparing original with recommended ones

In [21]:
FILE_NAMES = [
    "grandmaster_result.json",
    "master_result.json",
    "expert_result.json",
]

for name in FILE_NAMES:
    path = f"recommendation result/{name}"
    data_rank = name[:-12]

    with open(path, 'r') as file:
        original_file_path = open(f'data/{data_rank}_nl_pl_only_plot.json')
        original_data = json.load(original_file_path)

        data = json.load(file)

        print(f"rank {data_rank}: {len(data)} items || original : {len(original_data)} items")

C:\Users\Veerakit_Admin\AppData\Local\Temp\ipykernel_12900\2423600966.py:12: ResourceWarning: unclosed file <_io.TextIOWrapper name='data/expert_nl_pl_only_plot.json' mode='r' encoding='cp874'>
  original_file_path = open(f'data/{data_rank}_nl_pl_only_plot.json')
C:\Users\Veerakit_Admin\AppData\Local\Temp\ipykernel_12900\2423600966.py:12: ResourceWarning: unclosed file <_io.TextIOWrapper name='data/grandmaster_nl_pl_only_plot.json' mode='r' encoding='cp874'>
  original_file_path = open(f'data/{data_rank}_nl_pl_only_plot.json')


rank grandmaster: 2670 items || original : 2670 items
rank master: 4098 items || original : 4098 items


C:\Users\Veerakit_Admin\AppData\Local\Temp\ipykernel_12900\2423600966.py:12: ResourceWarning: unclosed file <_io.TextIOWrapper name='data/master_nl_pl_only_plot.json' mode='r' encoding='cp874'>
  original_file_path = open(f'data/{data_rank}_nl_pl_only_plot.json')


rank expert: 10851 items || original : 10855 items


In [44]:
output = {}
metadata = {
    
}
for name in FILE_NAMES:
    path = f"recommendation result/{name}"
    data_rank = name[:-12]

    with open(path, 'r') as file:
        original_file_path = open(f'data/{data_rank}_nl_pl_only_plot.json')
        original_data = json.load(original_file_path)
        data = json.load(file)
        print(f"rank {data_rank}: {len(data)} items || original : {len(original_data)} items")

        comparison_collection = {
            "ml": 0,
            "es": 0,
            "es-processed": 0,
        }
        incorrect_summarize = {
            # "ml": 0,
            "es": 0,
            # "es-processed": 0,
        }

        incorrect_pairs = []
        incorrect = {

        }
        i = 0
        count_empty = 0
        for item in data:
            if (not item['original_md']):
                count_empty = count_empty + 1
                continue

            original_code = item['original_code']
            ml_code = item['recommended_code_ml']
            es_base_code = item['recommended_code_es_base']
            es_processed_code = item['recommended_code_es_processed']

            if (item['original_code'] == item['recommended_code_ml']):
                comparison_collection["ml"] = comparison_collection["ml"] + 1

            if (item['original_code'] == item['recommended_code_es_processed']):
                comparison_collection["es-processed"] = comparison_collection["es-processed"] + 1
                
            if (item['original_code'] == item['recommended_code_es_base']):
                comparison_collection["es"] = comparison_collection["es"] + 1
                
            if (item['original_code'] != item['recommended_code_es_base']):
                incorrect_summarize["es"] = incorrect_summarize["es"] + 1
                incorrect_pairs.append({
                    "originalMarkdown": item['original_md'],
                    "originalCode": item['original_code'],
                    "bm25Code": item['recommended_code_es_base']
                })
        
        print(f"Skipped {count_empty} empty Markdown items in rank {data_rank}")
        
        output[data_rank] = {
            "correct_pairs_summarize": comparison_collection,
            "incorrect_summarize": {
                "es": len(data)-count_empty-comparison_collection["es"]
            },
            "incorrect_pairs_items": incorrect_pairs,
        }

        metadata[data_rank] = {
            "total_items": len(data),
            "total_skipped_empty_markdown": count_empty,
            "total_es_correct_pairs_": comparison_collection["es"],
            "total_es_incorrect_pairs_": incorrect_summarize["es"]
        }
        
with open(f"comparison_result.json", "w") as file:
    output = json.dump(output, file)
    print(f'successfully write to /result/comparison_result.json')
print("Done!!")

print(f"""
Summarize:

Rank Grandmaster
- Total items: {metadata['grandmaster']["total_items"]}
- Total skipped empty Markdown: {metadata['grandmaster']["total_skipped_empty_markdown"]}
# Below is only for Elasticsearch base approach #
- Total correct original-recommended pairs: {metadata['grandmaster']["total_es_correct_pairs_"]}
- Total incorrect original-recommended pairs: {metadata['grandmaster']["total_es_incorrect_pairs_"]}

Rank Master
- Total items: {metadata['master']["total_items"]}
- Total skipped empty Markdown: {metadata['master']["total_skipped_empty_markdown"]}
# Below is only for Elasticsearch base approach #
- Total correct original-recommended pairs: {metadata['master']["total_es_correct_pairs_"]}
- Total incorrect original-recommended pairs: {metadata['master']["total_es_incorrect_pairs_"]}

Rank Expert
- Total items: {metadata['expert']["total_items"]}
- Total skipped empty Markdown: {metadata['expert']["total_skipped_empty_markdown"]}
# Below is only for Elasticsearch base approach #
- Total correct original-recommended pairs: {metadata['expert']["total_es_correct_pairs_"]}
- Total incorrect original-recommended pairs: {metadata['expert']["total_es_incorrect_pairs_"]}
""")



C:\Users\Veerakit_Admin\AppData\Local\Temp\ipykernel_12900\3344088031.py:10: ResourceWarning: unclosed file <_io.TextIOWrapper name='data/expert_nl_pl_only_plot.json' mode='r' encoding='cp874'>
  original_file_path = open(f'data/{data_rank}_nl_pl_only_plot.json')


rank grandmaster: 2670 items || original : 2670 items
Skipped 22 empty Markdown items in rank grandmaster
rank master: 4098 items || original : 4098 items
Skipped 116 empty Markdown items in rank master


C:\Users\Veerakit_Admin\AppData\Local\Temp\ipykernel_12900\3344088031.py:10: ResourceWarning: unclosed file <_io.TextIOWrapper name='data/grandmaster_nl_pl_only_plot.json' mode='r' encoding='cp874'>
  original_file_path = open(f'data/{data_rank}_nl_pl_only_plot.json')
C:\Users\Veerakit_Admin\AppData\Local\Temp\ipykernel_12900\3344088031.py:10: ResourceWarning: unclosed file <_io.TextIOWrapper name='data/master_nl_pl_only_plot.json' mode='r' encoding='cp874'>
  original_file_path = open(f'data/{data_rank}_nl_pl_only_plot.json')


rank expert: 10851 items || original : 10855 items
Skipped 528 empty Markdown items in rank expert
successfully write to /result/comparison_result.json
Done!!

Summarize:

Rank Grandmaster
- Total items: 2670
- Total skipped empty Markdown: 22
# Below is only for Elasticsearch base approach #
- Total correct original-recommended pairs: 2512
- Total incorrect original-recommended pairs: 136

Rank Master
- Total items: 4098
- Total skipped empty Markdown: 116
# Below is only for Elasticsearch base approach #
- Total correct original-recommended pairs: 3572
- Total incorrect original-recommended pairs: 410

Rank Expert
- Total items: 10851
- Total skipped empty Markdown: 528
# Below is only for Elasticsearch base approach #
- Total correct original-recommended pairs: 9187
- Total incorrect original-recommended pairs: 1136



Filter and take some sample of data for analysis

In [45]:
import random 
import json
RANKS = ['grandmaster', 'master', 'expert']
MAX_SAMPLE = 30

with open("comparison_result.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    for rank in RANKS:
        incorrect_pairs = data[rank]['incorrect_pairs_items']
        with open(f'./analysis-result/{rank}_analysis.txt', 'w', encoding="utf-8") as file:
            random_range = list(range(0, len(incorrect_pairs)))
            random.shuffle(random_range)

            i = 0
            while i<MAX_SAMPLE:
                if (i>=MAX_SAMPLE):
                    break
                file.write(f"ITEM #{i+1}\n\n")
                file.write(f"ORIGINAL MARKDOWN:\n{'-'*99}\n")
                file.write(str(incorrect_pairs[i]['originalMarkdown']))
                file.write("\n================================================================\n")
                file.write("RECOMMENDED CODE:\n----------------------------------------------------\n")
                file.write(str(incorrect_pairs[i]['bm25Code']))
                file.write("\n================================================================\n")
                file.write("ORIGINAL CODE:\n----------------------------------------------------\n")
                file.write(str(incorrect_pairs[i]['originalCode']))
                file.write("\n" + "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n"*3 + "\n")

                i = i+1

    